In [1]:
import pandas as pd
import jutil

In [2]:
df = jutil.load_pd_df("select * from view_stu_course_mark where course_type='必' and speciality_code='0101' and student_id like '2013%'")
df.head()

,college_code,course_code,course_name,course_type,grade,mark,pmark,speciality_code,speciality_name,student_id,term_order
0,01,0100812,毕业实习与毕业设计（土木）,必,2013,8,82.0,0101,土木工程,20130002,5
1,01,0137400,工程流体力学,必,2013,8,82.0,0101,土木工程,20130002,4
2,01,0171079,基础工程A,必,2013,1,60.0,0101,土木工程,20130002,4
3,01,0472006,大学计算机基础A,必,2013,4,69.0,0101,土木工程,20130002,3
4,01,1111220,工科数学分析MII,必,2013,0,56.0,0101,土木工程,20130002,2


In [3]:
data = df[~df.course_name.str.startswith("形势与政策")]
data = data[~data.course_name.str.startswith("军事技能")]
data = data[~data.course_name.str.startswith("土木工程认识实习")]
data.count()

college_code       30410
course_code        30410
course_name        30410
course_type        30410
grade              30410
mark               30410
pmark              30410
speciality_code    30410
speciality_name    30410
student_id         30410
term_order         30410
dtype: int64

In [4]:
dg = data.groupby('student_id').groups

input_list = []
iii = None
out = data['college_code'].count()
print(out)
try:
    for k in dg:
        ids = dg[k]
        iii = ids
        items = []
        for id in ids:
            if id<out:
                items.append(data.iloc[id].get('course_code')+'_'+str(data.iloc[id].get('mark')))
        items = list(set(items)) # 去重
        input_list.append((k,items))
except:
    print(iii)
len(input_list)

30410


535

In [5]:
from pyspark.ml.fpm import FPGrowth

ddd = spark.createDataFrame(input_list,['id','items'])

fpG = FPGrowth(itemsCol="items",minSupport=0.05,minConfidence=0.6)
model = fpG.fit(ddd)

In [6]:
model.freqItemsets.show(n=10,truncate=False)

+---------------------------------+----+
|items                            |freq|
+---------------------------------+----+
|[8010220_9]                      |67  |
|[8010220_9, 6321910_0]           |30  |
|[8010220_9, 9990004_9]           |31  |
|[8010220_9, 0100812_9]           |27  |
|[8010220_9, 9991999_9]           |41  |
|[8010220_9, 9991999_9, 9992056_9]|29  |
|[8010220_9, 6321920_0]           |27  |
|[8010220_9, 9992056_9]           |46  |
|[6322020_4]                      |45  |
|[6322020_4, 9991999_9]           |27  |
+---------------------------------+----+
only showing top 10 rows



In [7]:
model.associationRules.show(truncate=False)

+-----------------------+-----------+------------------+
|antecedent             |consequent |confidence        |
+-----------------------+-----------+------------------+
|[0134300_10]           |[9992056_9]|0.6509433962264151|
|[3200502_9]            |[9991999_9]|0.6078431372549019|
|[3200502_9]            |[9992056_9]|0.6274509803921569|
|[8010230_4]            |[9992056_9]|0.6166666666666667|
|[6111010_7]            |[9992056_9]|0.6590909090909091|
|[9991633_9, 6321910_0] |[9992056_9]|0.7121212121212122|
|[0171060_9]            |[9991999_9]|0.7090909090909091|
|[9010130_9]            |[9991999_9]|0.6571428571428571|
|[9010140_7]            |[9992056_9]|0.6222222222222222|
|[7001063_7, 9991999_9] |[9992056_9]|0.6122448979591837|
|[6111010_0]            |[9992056_9]|0.6082474226804123|
|[6011320_0, 9990005_9] |[9992056_9]|0.6086956521739131|
|[6011310_10, 9992056_9]|[9991999_9]|0.6153846153846154|
|[7001063_8, 6321920_0] |[9992056_9]|0.85              |
|[9990005_7, 9991999_9] |[99920

In [8]:
pdc = model.associationRules.toPandas()
pdc.head()

,antecedent,consequent,confidence
0,[0134300_10],[9992056_9],0.650943
1,[3200502_9],[9991999_9],0.607843
2,[3200502_9],[9992056_9],0.627451
3,[8010230_4],[9992056_9],0.616667
4,[6111010_7],[9992056_9],0.659091


In [9]:
pdc.count()

antecedent    257
consequent    257
confidence    257
dtype: int64

In [10]:
# 提取出课程代码和课程名称的对应
course_arr = data.loc[:,['course_code','course_name']].values
course_dict = {}
for row in course_arr:
    course_dict[row[0]] = row[1]
len(course_dict)

411

In [11]:
antecedent = pdc['antecedent']
consequent = pdc['consequent']

In [12]:
consequent_name = [course_dict.get(row[0][:row[0].index('_')],'unknown') for row in consequent ]

In [13]:
antecedent_name = []
for row in antecedent:
    antecedent_name.append([course_dict.get(code[:code.index('_')],'unknown') for code in row])

In [14]:
for i in range(40):
    print('%s-->%s'%(antecedent_name[i],consequent_name[i]))

['基础工程B']-->土木工程概论（认识实习）
['大学计算机基础']-->课外创新实践
['大学计算机基础']-->土木工程概论（认识实习）
['英语Ⅲ']-->土木工程概论（认识实习）
['大学物理AⅠ']-->土木工程概论（认识实习）
['生产实习', '理论力学AⅠ']-->土木工程概论（认识实习）
['钢结构设计原理']-->课外创新实践
['体育Ⅲ']-->课外创新实践
['体育Ⅳ']-->土木工程概论（认识实习）
['中国近现代史纲要', '课外创新实践']-->土木工程概论（认识实习）
['大学物理AⅠ']-->土木工程概论（认识实习）
['高等数学Ⅱ', '土木工程地质实习']-->土木工程概论（认识实习）
['高等数学Ⅰ', '土木工程概论（认识实习）']-->课外创新实践
['中国近现代史纲要', '理论力学AⅡ']-->土木工程概论（认识实习）
['土木工程地质实习', '课外创新实践']-->土木工程概论（认识实习）
['大学物理实验Ⅰ', '理论力学AⅡ']-->土木工程概论（认识实习）
['毕业实习与毕业设计（土木）', '课外创新实践']-->土木工程概论（认识实习）
['军事理论']-->土木工程概论（认识实习）
['大学计算机基础']-->课外创新实践
['高等数学Ⅰ']-->土木工程概论（认识实习）
['体育Ⅳ', '生产实习']-->土木工程概论（认识实习）
['中国近现代史纲要', '理论力学AⅠ']-->土木工程概论（认识实习）
['中国近现代史纲要']-->土木工程概论（认识实习）
['体育Ⅰ', '理论力学AⅠ']-->土木工程概论（认识实习）
['工程测量实习', '土木工程概论（认识实习）']-->课外创新实践
['高等数学Ⅰ', '课外创新实践']-->土木工程概论（认识实习）
['英语Ⅲ']-->课外创新实践
['英语Ⅳ']-->课外创新实践
['英语Ⅳ']-->土木工程概论（认识实习）
['工程测量AⅠ']-->土木工程概论（认识实习）
['思想道德修养与法律基础', '理论力学AⅡ']-->课外创新实践
['思想道德修养与法律基础', '理论力学AⅡ']-->土木工程概论（认识实习）
['工程测量实习', '理论力学AⅡ']-->课外创新实践
['土木工程制图III-A', '课外创新实践']-->土木工